In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/GitHub/SSM")

include("/Users/ryansenne/Documents/GitHub/SSM//src/SSM.jl")
using .SSM
using LinearAlgebra
using Random
using Distributions
using Plots
using CSV
using DataFrames
using BenchmarkTools

  Activating project at `\Users\ryansenne\Documents\GitHub\SSM`

In [ ]:
# Generate data from a known dynamical system--a pendulum
g = 9.81 # gravity
l = 1.0 # length of pendulum
dt = 0.001 # time step
T = 0.004 # total time
# Discrete-time dynamics
A = [1.0 dt; -g/l*dt 1.0]
# Initial state
x0 = [0.0; 1.0]
# Time vector
t = 0:dt:T
# Generate data
x = zeros(2, length(t))
x[:,1] = x0
for i = 2:length(t)
    x[:,i] = A*x[:,i-1]
end

# Now lets add noise to the system and see if we can recover the dynamics
# Add driving Gaussian noise to simulate observations
x_noisy = zeros(2, length(t))
x_noisy[:, 1] = x0
observation_noise_std = 0.5

noise = rand(Normal(0, observation_noise_std), (2, length(t)))

for i in 2:length(t)
    x_noisy[:, i] = A * x[:, i-1] + noise[:, i]
end

# Define the LDS model parameters
H = I(2)  # Observation matrix (assuming direct observation)
Q = 1e-8 * I(2)  # Process noise covariance
R = 0.25 * I(2)  # Observation noise covariance
P0 = 1e-2 * I(2)  # Initial state covariance
x0 = [0.0; 1.0]  # Initial state mean

# Create the Kalman filter parameter vector
kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, ector([true, true, false, true, true, true, true, false]))

# Run the Kalman filter
x_filt, p_filt, x_pred, p_pred, v, F, K, ll = KalmanFilter(kf, Matrix{Float64}(x_noisy'))
# Run the smoother
x_smooth, P_smooth = KalmanSmoother(kf, Matrix{Float64}(x_noisy'))

In [9]:
# calculate the predictions of X
using Profile
x_prediction = zeros(length(t), 2)
x_prediction[1, :] = kf.x0
for t in 2:length(t)
    x_prediction[t, :] = kf.A * x_prediction[t-1, :]
end

@profile x_new = SSM.newton_raphson_tridg!(kf, x_prediction, x_noisy', 10, 1e-3)

UndefVarError: UndefVarError: `kf` not defined

In [13]:
using ForwardDiff
# H = ForwardDiff.hessian(x -> SSM.loglikelihood(x', kf, x_noisy[1:2, 1:100]'), x_prediction')
G = ForwardDiff.gradient(x -> SSM.loglikelihood(x', kf, x_noisy'), x_new')
G = reshape(G, 10, 1)

UndefVarError: UndefVarError: `x_new` not defined

In [3]:
# need to use previously aquired pendulum data for comparison
pendulum_data = CSV.read("/Users/ryansenne/Documents/GitHub/SSM/notebooks/pendulum_data_1.csv", DataFrame)

# convert x_noisy columns to matrix for use 
x_noisy =  Matrix(pendulum_data[1:10, 3:4])

# now create a Kalman Filter where we want to learn the parameters
A = [0.1 0.2; 0.3 0.4]
H = [1 0.0; 0.0 1]
Q = 1 * I(2)
R = 1 * I(2)
p0 = 1 * I(2)
x0 = [0.0, 1.0]

# kf = LDS(A, Matrix{Float64}(H), nothing, Matrix{Float64}(Q), Matrix{Float64}(R), x0, Matrix{Float64}(P0), nothing, 2, 2, "Gaussian", Vector([true, true, false, true, true, true, true, false]))
kf_unknown = LDS(A, H, nothing, Q, R, x0, p0, nothing, 2, 2, [true, true, true, true, true, true])
# SSM.pca_init!(kf_unknown, x_noisy)

SSM.KalmanFilterEM!(kf_unknown, Matrix{Float64}(x_noisy), 100)

Log-likelihood at iteration 1: -18.52416478401385
Log-likelihood at iteration 2: -8.19497836514248
Log-likelihood at iteration 3: -6.176857584849762
Log-likelihood at iteration 4: -4.58278965181124
Log-likelihood at iteration 5: -3.551206312693214
Log-likelihood at iteration 6: -2.926615373182244
Log-likelihood at iteration 7: -2.536716907196628
Log-likelihood at iteration 8: -2.2734743110956552
Log-likelihood at iteration 9: -2.0807815795973506
Log-likelihood at iteration 10: -1.9308942394226316
Log-likelihood at iteration 11: -1.8096566052906937
Log-likelihood at iteration 12: -1.7092011456132208
Log-likelihood at iteration 13: -1.6246400726833459
Log-likelihood at iteration 14: -1.552621178739051
Log-likelihood at iteration 15: -1.490687941005494
Log-likelihood at iteration 16: -1.436970194145072
Log-likelihood at iteration 17: -1.3900126314579275
Log-likelihood at iteration 18: -1.3486666845015916
Log-likelihood at iteration 19: -1.3120162841336418
Log-likelihood at iteration 20: -

Log-likelihood at iteration 30: -1.0788542488785429
Log-likelihood at iteration 31: -1.0662856578660231
Log-likelihood at iteration 32: -1.0545413601263152
Log-likelihood at iteration 33: -1.0435441764190334
Log-likelihood at iteration 34: -1.0332267039026977
Log-likelihood at iteration 35: -1.023529782568204
Log-likelihood at iteration 36: -1.0144012339294317
Log-likelihood at iteration 37: -1.0057948200028524
Log-likelihood at iteration 38: -0.997669381087321
Log-likelihood at iteration 39: -0.9899881190977741
Log-likelihood at iteration 40: -0.9827179997228972
Log-likelihood at iteration 41: -0.9758292518521738
Log-likelihood at iteration 42: -0.9692949468435331
Log-likelihood at iteration 43: -0.9630906435044718
Log-likelihood at iteration 44: -0.9571940873088075
Log-likelihood at iteration 45: -0.9515849545033669
Log-likelihood at iteration 46: -0.9462446334789757
Log-likelihood at iteration 47: -0.9411560371704468
Log-likelihood at iteration 48: -0.9363034413764818
Log-likelihood

-0.8591873068551975
Log-likelihood at iteration 76: -0.8575190896979759
Log-likelihood at iteration 77: -0.8559000665664526
Log-likelihood at iteration 78: -0.854327980098869
Log-likelihood at iteration 79: -0.8528006927596395
Log-likelihood at iteration 80: -0.8513161795791788
Log-likelihood at iteration 81: -0.8498725213866228
Log-likelihood at iteration 82: -0.8484678984977493
Log-likelihood at iteration 83: -0.8471005848239849
Log-likelihood at iteration 84: -0.845768942370819
Log-likelihood at iteration 85: -0.8444714160971692
Log-likelihood at iteration 86: -0.84320652910926
Log-likelihood at iteration 87: -0.8419728781648115
Log-likelihood at iteration 88: -0.8407691294656646
Log-likelihood at iteration 89: -0.8395940147181101
Log-likelihood at iteration 90: -0.8384463274423334
Log-likelihood at iteration 91: -0.8373249195137009
Log-likelihood at iteration 92: -0.8362286979198557
Log-likelihood at iteration 93: -0.835156621718882
Log-likelihood at iteration 94: -0.83410769918505

(LDS([-0.0071167124470350035 -0.03494928974252329; 0.05097519773929564 0.9852158737691314], [0.004188002858171138 -0.1691203704740746; 0.03222372521369811 0.5626550385423239], nothing, [0.5206899860293644 -0.026369212673853275; -0.026369212673853275 0.0043013779045373785], [0.2845173964416663 0.05839691137905212; 0.05839691137905212 0.13085478563920255], [0.0, 1.0], [1 0; 0 1], nothing, 2, 2, Bool[0, 0, 1, 1, 1, 1]), -0.8282504134054396)

In [15]:
x_filt, p_filt, x_pred, p_pred, v, S, K, ll =SSM.KalmanFilter(kf_unknown, Matrix{Float64}(x_noisy))

([0.0 1.0; 0.22566239984008948 0.45610640028696026; … ; -0.08326931777853783 -0.04415219514174039; -0.015233336622534323 0.0071783586296058285], [0.5 0.0; -0.02159198891631108 -0.006507611970760366; … ; -0.06016781257525112 -0.0807295298198789; -0.060184103980500656 -0.08076049549473423;;; 0.0 0.5; -0.006507611970760364 0.1163386778832134; … ; -0.0807295298198789 -0.02941233463191853; -0.08076049549473423 -0.029471192232264215], [0.0 1.0; 0.2 0.4; … ; -0.01461213073443553 -0.034279829983049084; -0.01715737080620186 -0.042641673390257505], [1.0 0.0; -0.02108970643722805 -0.007209060966286943; … ; -0.05108643781697929 -0.07441658035128658; -0.05109705914105246 -0.07444003509608585;;; 0.0 1.0; -0.007209060966286943 0.1317083691926869; … ; -0.07441658035128658 -0.0227360002420455; -0.07444003509608585 -0.0227877946369636], [0.0 0.0; -1.3117510767744314 0.4088926756141953; … ; -0.25747359019323585 1.0423547610109465; -0.8353662184963082 0.5987052565231548], [2.0 0.0; 0.9789102935627719 -0.0

In [28]:
for i in 1:10
    z = ((kf_unknown.H * p_pred[i, :, :] * kf_unknown.H') + kf_unknown.R)
    println(det(z))
end

4.0
1.1077890013538432
0.9760529538224801
0.9398591439616736
0.9280005251778541
0.9239155790544029
0.9224841152765938
0.9219794969436195
0.9218012357723678
0.9217382166871365


In [21]:
p_filt[4, :, :]

2×2 Matrix{Float64}:
 -0.0555903  -0.072028
 -0.072028   -0.0128714

In [3]:
kf_unknown.R

2×2 Matrix{Float64}:
  0.300558  -0.170841
 -0.170841   0.387198